<a href="https://colab.research.google.com/github/sbuergers/llm-hackathon/blob/master/llm_challenge_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Large Language Models Challenge!

This is a template notebook. __Copy it__ and start hacking away if you like. We suggest removing the `_template` suffix and replacing it with your team name. If you do not have a team name yet, have a look below. There are already a few useful snippets of code that might help you in your quest, including a random team name generator!

In [3]:
import random

# Function to generate a random team name
def generate_team_name():
    adjectives = ["Agile", "Brave", "Clever", "Daring", "Energetic", "Fearless", "Gallant", "Heroic", "Innovative", "Jovial", "Keen", "Loyal", "Mighty", "Noble", "Optimistic", "Persistent", "Quick", "Resilient", "Strong", "Tenacious", "Unyielding", "Valiant", "Wise", "Xenial", "Youthful", "Zealous"]
    nouns = ["Antelopes", "Bears", "Cheetahs", "Dolphins", "Elephants", "Foxes", "Giraffes", "Hawks", "Iguanas", "Jaguars", "Kangaroos", "Lions", "Monkeys", "Nightingales", "Owls", "Penguins", "Quails", "Rabbits", "Snakes", "Tigers", "Unicorns", "Vultures", "Wolves", "Xiphosuran", "Yaks", "Zebras"]
    adjective = random.choice(adjectives)
    noun = random.choice(nouns)
    return adjective.lower() + "_" + noun.lower()

# Generate a random team name
team_name = generate_team_name()
print("Generated team name: ", team_name)


Generated team name:  noble_monkeys


##Getting results from the classical machine learning model

There are 5 models, one for each level of the hierarchy. You can select which one to query using the `level` parameter in the `get_model_output` function. Also have a look at the other parameters you can pass.

In [6]:
import requests

FUN_URL = "https://cpvclassifier.azurewebsites.net/api/azfun_predict_cpv_light"


def get_model_output(tender_text: str, level: str, num_pred: int, service_filter: str) -> dict:
    """Get model output from API endpoint (see cpv repository for details).

    Parameters
    ----------
    tender_text: str
        The text for which to make predictions
    level: str
        The level at which to run the model (e.g. department or group).
    num_pred : int
        How many predictions to show (top `num_pred` predictions are shown)
    service_filter : str
        Users can select only `diensten` (services), `werken` (works),
        `leveringen` (deliveries), or `diensten, leveringen, werken` (all).

    Returns
    -------
    dict
        A dictionary with the model output, can be coerced into a dataframe.
    """
    payload = {
        "tender_text": tender_text,
        "level": level,
        "num_predictions": num_pred,
        "service_filter": service_filter,
    }
    response = requests.post(FUN_URL, params=payload)
    output_dict = response.json()

    return output_dict


tender_text = "Dit document is een officiële uitnodiging tot inschrijving voor de bouw van een nieuwe brug. De CPV-code voor deze aanbesteding is 45221100, wat overeenkomt met bruggenbouw en -herstel. Wij nodigen gekwalificeerde en ervaren bedrijven uit om hun voorstellen in te dienen voor dit spannende project."
level = "Omschrijving"  # From highest to lowest: 'afdeling', 'groep', 'categorie', 'klasse', 'Omschrijving'"
results_dict = get_model_output(
    tender_text=tender_text,
    level=level,
    num_pred=5,
    service_filter=None,
)
print(results_dict.get("results"))

{'CPV-code': {'631': '45233120-6', '646': '79000000-4', '474': '71242000-6', '130': '45233100-0', '527': '90919300-5'}, 'Omschrijving': {'631': 'Wegenbouwwerken', '646': 'Zakelijke dienstverlening: juridisch, marketing, consulting, drukkerij en beveiliging', '474': 'Project- en ontwerpvoorbereiding, kostenraming', '130': 'Bouwwerkzaamheden voor hoofdwegen en wegen', '527': 'Schoonmaken van scholen'}, 'Score': {'631': 0.002438464944395204, '646': 0.0021191854887598893, '474': 0.0018482626442439531, '130': 0.001817809503174387, '527': 0.0017991071263125358}, 'Type opdracht': {'631': 'werken', '646': 'diensten', '474': 'diensten', '130': 'werken', '527': 'diensten'}, 'important_words': {'631': 'bruggenbouw, officieel, project, spannend, in_dienen', '646': 'spannend, overeen_komen, project, bruggenbouw, bedrijf', '474': 'spannend, voorstel, inschrijving, project, nodig', '130': 'bruggenbouw, overeen_komen, brug, voorstel, nieuw', '527': 'officieel, ervaren, spannend, voorstel, in_dienen'}}

## Why not scrape some additional info from the web?

In [1]:
from bs4 import BeautifulSoup
import requests

# Function to extract first text paragraph after first h2 tag from website.
# (If you use https://cpvcodes.eu/en/{cpv_code}-cpv/, you get a detailed description for that code)
def extract_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the first <h2> tag
    first_h2 = soup.find('h2')

    if first_h2:
        # Find the next <p> tag after the first <h2> tag
        first_p_after_h2 = first_h2.find_next('p')
        if first_p_after_h2:
            return first_p_after_h2.text
        else:
            return "No <p> tag found after the first <h2> tag."
    else:
        return "No <h2> tag found in the webpage."


# Test the function
url = "https://cpvcodes.eu/en/03211000-cpv/"
print(extract_text(url))

The Cereals category includes various types of grains that are commonly used as food sources. These grains are essential ingredients in many food products and are widely consumed worldwide. The subcategories within this category consist of Wheat, Maize (corn), Rice, Barley, Rye, Oats, Malt, and Grain products. Wheat is a versatile grain used in bread, pasta, and pastries. Maize is primarily used for animal feed and as a raw material for various food products. Rice is a staple food in many cultures and is consumed in various forms. Barley is often used in brewing and as a nutritious grain. Rye and oats are commonly used in bread, cereals, and other baked goods. Malt is a key ingredient in beer production. Grain products encompass a wide range of processed goods derived from grains, such as flour, cereals, and snacks.
